In [1]:
import pandas as pd
import numpy as np
import openai
import json

# # our groupings
# seed_hashtags = pd.read_csv("community-grouping_20240809.csv")
# # all hashtags double hit
# dat = pd.read_csv('double_hits_edges_no_dupes.csv')

# open the secrets file
with open('../data/secrets.json') as f:
    secrets = json.load(f)

api_key = secrets['OPENAI_API_KEY_SR2']

client = openai.Client(api_key=api_key)

/Users/laylabouzoubaa/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/laylabouzoubaa/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [57]:
# check rate limits
def check_rate_limits(api_key):
    openai.api_key = api_key

    try:
        # Make a chat completion API call with raw response
        api_response = client.chat.completions.with_raw_response.create(
            model="gpt-4o",  
            messages=[
                {
                    "role": "user",
                    "content": "Hello, this is a test message to check rate limits.",
                }
            ],
        )

        # Access the headers
        headers = api_response.headers

        # Print rate limit information
        print("Rate Limit Information:")
        print(f"Remaining Requests: {headers.get('x-ratelimit-remaining-requests')}")
        print(f"Remaining Tokens: {headers.get('x-ratelimit-remaining-tokens')}")
        print(f"Reset Tokens: {headers.get('x-ratelimit-reset-tokens')}")
        print(f"Reset Requests: {headers.get('x-ratelimit-reset-requests')}")

        # Print the response content
        response = api_response.parse()
        print(f"\nResponse content: {response.choices[0].message.content}")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [58]:
check_rate_limits(api_key)

Rate Limit Information:
Remaining Requests: 9999
Remaining Tokens: 1999970
Reset Tokens: 0s
Reset Requests: 6ms

Response content: Hello! Feel free to continue messaging if you have any questions or need information. I'm here to help.


---
## IRR reliability with human annotation

In [2]:
def convert_to_tags(input_string):
    # Split the input string into lines
    lines = input_string.strip().split('\n')
    
    # Process each line
    tags = []
    for line in lines:
        # Split each line into words
        words = line.split()
        # Add each word as a tag
        tags.extend(f'{word}' for word in words)
    
    # # Join the tags with commas
    # result = ', '.join(tags)
    
    # Count the total number of tags
    tag_count = len(tags)
    
    return tags, tag_count

# Example input
input_string = """funny	love	depressionandanxiety	alcohol	weed	cognitiveenhancers	opioids	smoking	harmreductionsaveslives	vitamins	fyp	stoned	vaping	kensingtonphilly	medstudent	lgbtqia
addictionhumor	hope	addiction	beers	pot	modafinil	opiates	drinking	soberlife	creatine	greenscreen	high	nicotine	vancouverbc	nurselife	transgender
meme	struggle	opioidaddiction	alchohol	cannabis	nootropics	heroin	drink	druglaws	magnesium	tiktok	wasted	ecig	bayarea	exprostitute	lgbtqtiktok
	kindness	substanceabuse	vodka	greens	nootrostatic	fent	shmokesumthin	drugpolicy	magnesiumglycinate	viral	blackedout	ecigs	toronto	medicalstudent	queergirl
	lovemyself	substanceusedisorder	whiskey	stonersoftiktok	smartdrugs	fentfriday	shmoke	pwud	vitamink	trending	highasfuck	juul	philly	healthcareworkers	queertiktok
	happytobealive	mentalhealth	cocktails	stonervibes	smartpills	xans	shmoketok	pwuds	tylenol	capcut	haf	cigarettes	midwest	socialworkersoftiktok	feralqueer
	trauma	chronicpain	wine	stoner	cerebrolysin	xanny	keepsmokingtheherb	opioidawareness	disprin	duet	shitfaced	vaper	boston	socialworker	whitegirl
	loveyall	chronicpainsufferers	liquor	laganjaestranja		xantok	drinkingtools	opioidcrisis	ayurveda	stitch	tipsy	cigarette	sanfrancisco	frontlineworkers	transtok
	vibes	headache	tequila	420vibes		xandemic	injection	overdoseawareness	tylenoltuesday	fyp„Ç∑„Çöviral	drunk	vape	usa	matnurse	indian
	happy	migraine	booze	dabs		m0lly	injectinguser	overdosecrisis	ibuprofen	fyp„Ç∑	hammered	cig	uk	streetnurse	gaygirl
	gratitude	addicted	chugbeer	wax		molly	inhaler	harmreduction	suplements	foryou	blackout	cigs	canada	nursetiktok	
	fear	bpd	beerfunnel	shmeeds		percs	inhaling	harmreductionworks	coughsyrup	foryoupage	hightimes	tobacco		therapist	
	makesomeonesmile	bpdtiktok	drinkinggames			percoset	keepsmoking	harmreductiontips	overthecounter	foryourpage	gurning	nicotinefein			
		addictionisreal	wildmandrinking			perc30s	inhalation	narcan	ibuprofeno	xyzbca	gurner	nicotinegum			
		ptsd				lean		narcansaveslives	collagen	trend	drunkaf	nicorettepatch			
		depression				leantok		naloxone							
		anxiety				acid		naloxonesaveslives							
		chronicpainwarrior				mescaline		safesupply							
		allergy				pinger		endoverdose							
		infection				pingertok		overdoseresponse							
		allergies				pingtok		stopsmoking							
		seizures				pingerzzzz		smokingkills	addictionhumor
drughumor
recoveryhumor						
		epilespy				emma		quitnicotine							
		insomnia				drank		stopvapping							
		fever				thatdrank		quitvapping							
		cough				bars		sobriety							
		adtiction				30s		sober							
		asthma				40s		soberliving							
		sorethroat				512s		cleanandsober							
		flu				yellows		recovery							
		fluseason				biak		onedayatatime							
		bronchitis				epills		odaat							
		pain				psychedelictok		na							
		inhalantsaddiction				lsdart		detox							
						magicmushroomsadventures		sobrietybirthday							
						libcaps		soberhouse							
						libertycapss		wedorecover							
						thom		relapsehappens							
						nosebeers		methadoneclinic							
						c0deine		naltrexone							
						inhalants		drugrehab							
						whippets		dropthosedates							
						whipit		matsaves							
						balloons		stopthestigma							
						adderral		alcoholfreejourney							
						addy		alcoholic							
								alcoholism																								"""

# Convert and print the result
all_human_tags, count = convert_to_tags(input_string)
print("Converted tags:")
print(all_human_tags)
print(f"\nTotal number of tags: {count}")

Converted tags:
['funny', 'love', 'depressionandanxiety', 'alcohol', 'weed', 'cognitiveenhancers', 'opioids', 'smoking', 'harmreductionsaveslives', 'vitamins', 'fyp', 'stoned', 'vaping', 'kensingtonphilly', 'medstudent', 'lgbtqia', 'addictionhumor', 'hope', 'addiction', 'beers', 'pot', 'modafinil', 'opiates', 'drinking', 'soberlife', 'creatine', 'greenscreen', 'high', 'nicotine', 'vancouverbc', 'nurselife', 'transgender', 'meme', 'struggle', 'opioidaddiction', 'alchohol', 'cannabis', 'nootropics', 'heroin', 'drink', 'druglaws', 'magnesium', 'tiktok', 'wasted', 'ecig', 'bayarea', 'exprostitute', 'lgbtqtiktok', 'kindness', 'substanceabuse', 'vodka', 'greens', 'nootrostatic', 'fent', 'shmokesumthin', 'drugpolicy', 'magnesiumglycinate', 'viral', 'blackedout', 'ecigs', 'toronto', 'medicalstudent', 'queergirl', 'lovemyself', 'substanceusedisorder', 'whiskey', 'stonersoftiktok', 'smartdrugs', 'fentfriday', 'shmoke', 'pwud', 'vitamink', 'trending', 'highasfuck', 'juul', 'philly', 'healthcarewo

In [5]:
import random
import ast

# Read the CSV file
df = pd.read_csv('../data/kcore_10_nodes.csv')

# Extract the 'node' column and get the count
node_list = df['hashtag'].dropna().tolist()
node_count = len(node_list)
print(f"Total count of nodes: {node_count}")

# Remove occurrences of all_human_tags from node_list
node_list = [node for node in node_list if node not in all_human_tags]

# Get the count of remaining nodes
node_count = len(node_list)
print(f"Total count of nodes after removal: {node_count}")

# Use all remaining nodes instead of random sampling
print(f"Remaining nodes: {node_list}")

Total count of nodes: 2333
Total count of nodes after removal: 2150
Remaining nodes: ['peopleoverprofit', 'harmreductiontiktok', 'b', '4u', 'greenscreenvideo', 'signs', 'medications', 'us', 'virall', 'fentanyleducation', 'mentallyill', 'clips', 'nurse', 'alcoholaddiction', 'education', 'paradise', 'addicthumor', 'schizoaffective', 'amazon', 'bestie', 'addictedseries', 'motivational', 'crank', 'indigenoustiktok', 'pretty', 'adhd', 'البحرين', 'nonbinary', 'smallbusiness', 'hcvtesting', 'child', 'save', 'footballs', 'recoverycomedy', 'beach', 'juice', 'wellbriety', 'comedian', 'bipolarawareness', 'daily', 'louisiana', 'painadvocate', 'teens', 'vitaminr', 'addictionmedicine', 'losangeles', 'qatar', 'autism', 'retokfornature', 'endprohibition', 'chevy', 'trump', 'growthmindset', 'mexico', '❤️', 'cbd', 'five', 'invisibleillness', 'prohibitionispoison', 'strength', 'hazbinhotel', 'fitness', 'ventana', 'hard', 'painpatient', 'genshinimpact', 'fearmongering', 'firefighter', 'bowls', 'drugpolicy

In [6]:
# test new function
# from get_themes import get_theme
# import random
# hashtag_sample = random.sample(node_list, 100)
# hashtag_sample_str = ", ".join(hashtag_sample)

In [7]:
# hashtag_sample_df = pd.DataFrame(hashtag_sample)
# hashtag_sample_df.to_csv("../data/sample500_batch_4o.csv")

In [8]:
# Get themes for the sampled hashtags
# gpt4o : gpt-4o-2024-05-13
# gpt4o-mini: gpt-4o-mini-2024-07-18
# themes = get_theme(hashtag_sample_str, model= "gpt-4o-2024-05-13", client= client)

In [9]:
# print(themes)

In [10]:
# def process_themes(themes_string):
#     try:
#         themes_dict = json.loads(themes_string)
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON: {e}")
#         return pd.DataFrame(columns=['Category', 'Hashtag'])

#     # Step 3: Process the dictionary to create a pandas DataFrame
#     data = []
#     for category, hashtags in themes_dict.items():
#         for hashtag in hashtags:
#             data.append({'Category': category, 'Hashtag': hashtag})
    
#     df = pd.DataFrame(data)
#     return df


In [11]:
# themes_df = process_themes(themes)
# themes_df.to_csv("../data/themes100_gpt4o.csv")

---
## Experiment with parallelization

In [6]:
from pararallelize import process_posts_in_parallel
from get_themes import get_theme
import random

def chunk_list_to_df(input_list, chunk_size):
  """
  Chunks a list into smaller lists of approximately equal size and returns a DataFrame.

  Args:
      input_list: The list to be chunked.
      chunk_size: The desired size of each chunk.

  Returns:
      A DataFrame where each row contains a chunk of the original list.
  """
  chunks = []
  for i in range(0, len(input_list), chunk_size):
    chunks.append(input_list[i:i + chunk_size])

  df = pd.DataFrame({'hashtags': chunks})  
  df["hashtags_str"] = df["hashtags"].apply(lambda x: ', '.join(x))

  return df

# hashtag_sample = random.sample(node_list, 100)
hashtag_chunks_df = chunk_list_to_df(node_list, 500)


In [7]:
hashtag_chunks_df["themes"] = process_posts_in_parallel(hashtag_chunks_df["hashtags_str"].to_list(), max_workers=10,model = "gpt-4o-2024-05-13", task = get_theme, client = client)

In [8]:
hashtag_chunks_df.to_csv("../data/hashtag_themes_10kcore_2k_20241120.csv")

In [9]:
def convert_string_to_dataframe(input_string):
    # Parse the JSON string
    data = json.loads(input_string)
    
    # Create lists to store the data
    themes = []
    hashtags = []
    
    # Iterate through the dictionary
    for theme, tags in data.items():
        for tag in tags:
            themes.append(theme)
            hashtags.append(tag)
    
    # Create the DataFrame
    df = pd.DataFrame({
        'theme': themes,
        'hashtag': hashtags
    })
    
    return df

In [10]:
def process_hashtag_chunks(hashtag_chunks_df):
    all_hashtags_themes = pd.DataFrame()
    error_rows = []

    for index, row in hashtag_chunks_df.iterrows():
        try:
            theme = row["themes"]
            blah = convert_string_to_dataframe(theme)
            all_hashtags_themes = pd.concat([blah, all_hashtags_themes], ignore_index=True)
        except Exception as e:
            print(str(e))
            error_rows.append({
                "index": index,
                "theme": theme,
                "error": str(e)
            })
    
    return all_hashtags_themes, error_rows

In [11]:
result_df, errors = process_hashtag_chunks(hashtag_chunks_df)

Unterminated string starting at: line 7 column 11604 (char 12385)
Unterminated string starting at: line 7 column 6339 (char 7148)
Unterminated string starting at: line 5 column 9244 (char 9854)
Unterminated string starting at: line 9 column 11718 (char 12790)
Unterminated string starting at: line 67 column 278 (char 11735)


In [12]:
result_df.to_csv("../data/hashtag_themes_kcore2k_long.csv")
errors_df = pd.DataFrame(errors)
errors_df.to_excel("../data/hashtag_themes_kcore2k_errors.xlsx")

/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_30813/4204586226.py:3: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  errors_df.to_excel("../data/hashtag_themes_kcore2k_errors.xlsx")


---
TEST IF NON ASCII CHARACTERS ARE THE PROBLEM

In [13]:
import re

def is_clean_hashtag(text):
    """Check if hashtag contains only ASCII characters and basic punctuation"""
    return bool(re.match(r'^[a-zA-Z0-9_]+$', str(text)))

def sample_clean_hashtags(df, n=500, seed=42):
    """Sample n clean hashtags from dataframe"""
    # Mark clean hashtags
    df['is_clean'] = df['hashtag'].apply(is_clean_hashtag)
    
    # Sample from clean hashtags
    clean_df = df[df['is_clean']].copy()
    sample_df = clean_df.sample(n=min(n, len(clean_df)), random_state=seed)
    
    print(f"Total hashtags: {len(df)}")
    print(f"Clean hashtags: {len(clean_df)}")
    print(f"Sampled: {len(sample_df)}")
    
    # Create comma-separated string for GPT
    sample_string = ', '.join(sample_df['hashtag'].tolist())
    
    return sample_df, sample_string

# Sample usage
sample_df, gpt_input = sample_clean_hashtags(df)

Total hashtags: 2333
Clean hashtags: 2290
Sampled: 500


In [14]:
# Test with GPT
response = get_theme(gpt_input, model="gpt-4o-2024-05-13", client=client)

In [16]:
def validate_gpt_response(response):
    try:
        parsed = json.loads(response)
        return parsed, None
    except json.JSONDecodeError:
        print("Raw response:")
        print(response)
        return None, "JSON parsing failed"

# Test the response
parsed, error = validate_gpt_response(response)
if parsed:
    all_classified = [tag for tags in parsed.values() for tag in tags]
    duplicates = pd.Series(all_classified).value_counts()[pd.Series(all_classified).value_counts() > 1]
    print("\nDuplicates:", "None" if len(duplicates) == 0 else duplicates)
else:
    print("Error:", error)

# If you want to see the response structure:
print("\nFirst 500 characters of response:")
print(response[:500])

Raw response:
{
    "emotions and feelings": ["heartbroken", "helpme", "happytobealive", "vibes", "loved", "depressedtiktok", "laughing", "grief", "stress", "laughter", "sadtok", "struggle", "healingjourney", "goodmorning", "good", "loved", "depressionanxiety", "depressedtiktok", "laughing", "grief", "stress", "laughter", "sadtok", "struggle", "healingjourney", "goodmorning", "good"],
    "health conditions": ["chronicpain", "ptsd", "headaches", "bipolar", "schizophrenic", "bipolar1disorder", "add", "painpatient", "chronicpainwarrior", "intractablepain", "depressionhelp", "mentalhealthtok", "mentalhealthrecovery", "lupus", "fibromyalgia", "cripple", "ms"],
    "alcohol": ["vodka", "whiskey", "tequila", "cocktails", "wine", "booze", "beer"],
    "cannabis": ["weed", "stonersoftiktok", "dabs", "420vibes", "auntmary", "sinsemilla", "grasstok", "stoned", "puffpuffpass", "420", "420vibes", "grass"],
    "cognitive enhancement": ["modafinil", "nootropics", "piracetam", "cerebrolysin"],
    "

it seems like its donig fine up until this point (attached). i dont know why this behavior is happening. i have a couple of suspicions. maybe its the input length? it cant be non-ascii characters because we just tested that on 500 clean hashtags. maybe its the ask of returning responses in json format?


---

In [17]:
# try data with smaller chunk = 100
hashtag_chunks_df = chunk_list_to_df(node_list, 100)

In [21]:
def process_hashtags(hashtag_chunks_df):
    # Store responses for each chunk
    responses = []
    
    # Process each chunk and collect responses
    for i, chunk in enumerate(hashtag_chunks_df["hashtags"]):
        print(f'Processing chunk {i+1}/{len(hashtag_chunks_df["hashtags"])}')
        chunk_str = ', '.join(chunk)
        response = get_theme(chunk_str, model="gpt-4o-2024-05-13", client=client)
        responses.append(response)
        print(f"Response length: {len(response)}")
    
    # Parse all responses
    all_results = {}
    for i, response in enumerate(responses):
        try:
            parsed = json.loads(response)
            print(f"Successfully parsed chunk {i+1}, found {len(parsed)} categories")
            all_results.update(parsed)
        except json.JSONDecodeError as e:
            print(f"\nError parsing chunk {i+1}")
            print(f"Error message: {str(e)}")
            print(f"First 100 chars of problematic response: {response[:100]}")
            print(f"Last 100 chars of problematic response: {response[-100:] if len(response) > 100 else response}")
    
    print(f"\nTotal successful parses: {len(all_results)}")
    return all_results, responses

In [20]:
results2 = process_hashtags(hashtag_chunks_df)

Processing chunk 1/22
Error in chunk 1
First 100 chars of response: {
    "emotions and feelings": ["motivational", "strength", "depressed"],
    "health conditions": [
Processing chunk 2/22
Processing chunk 3/22
Processing chunk 4/22
Processing chunk 5/22
Processing chunk 6/22
Processing chunk 7/22
Processing chunk 8/22
Processing chunk 9/22
Processing chunk 10/22
Processing chunk 11/22
Error in chunk 11
First 100 chars of response: {
    "emotions and feelings": ["inspire", "anxietyrelief", "realtalk", "today"],
    "health condit
Processing chunk 12/22
Error in chunk 12
First 100 chars of response: {
    "emotions and feelings": ["missyou", "lovely"],
    "health conditions": ["bipolar", "addictio
Processing chunk 13/22
Error in chunk 13
First 100 chars of response: {
            "emotions and feelings": ["depressedtiktok", "loveislove", "relax", "inspiration", "se
Processing chunk 14/22
Processing chunk 15/22
Processing chunk 16/22
Processing chunk 17/22
Processing chunk 18/22
Erro

In [23]:
from get_themes_simple import get_theme_simple

def process_hashtags_batch(hashtags_df, batch_size=10):
    """Process hashtags in small batches"""
    results = {}
    total = len(hashtags_df)
    
    for i in range(0, total, batch_size):
        batch = hashtags_df['hashtag'].iloc[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(total+batch_size-1)//batch_size}")
        
        # Convert batch to comma-separated string
        batch_str = ', '.join(batch)
        
        # Get classifications
        response = get_theme_simple(batch_str, model="gpt-4o-2024-05-13", client=client)
        
        # Parse simple format (hashtag: category)
        for line in response.split('\n'):
            if ':' in line:
                hashtag, category = line.split(':')
                results[hashtag.strip()] = category.strip()
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(
        [(k, v) for k, v in results.items()], 
        columns=['hashtag', 'theme']
    )
    
    return results_df

In [24]:
batch_size = 10  # You can adjust this number
results_df3 = process_hashtags_batch(df, batch_size=10)

Processing batch 1/234
Processing batch 2/234
Processing batch 3/234
Processing batch 4/234
Processing batch 5/234
Processing batch 6/234
Processing batch 7/234
Processing batch 8/234
Processing batch 9/234
Processing batch 10/234
Processing batch 11/234
Processing batch 12/234
Processing batch 13/234
Processing batch 14/234
Processing batch 15/234
Processing batch 16/234
Processing batch 17/234
Processing batch 18/234
Processing batch 19/234
Processing batch 20/234
Processing batch 21/234
Processing batch 22/234
Processing batch 23/234
Processing batch 24/234
Processing batch 25/234
Processing batch 26/234
Processing batch 27/234
Processing batch 28/234
Processing batch 29/234
Processing batch 30/234
Processing batch 31/234
Processing batch 32/234
Processing batch 33/234
Processing batch 34/234
Processing batch 35/234
Processing batch 36/234
Processing batch 37/234
Processing batch 38/234
Processing batch 39/234
Processing batch 40/234
Processing batch 41/234
Processing batch 42/234
P

In [25]:
results_df3.to_csv("../data/themes_10core_20241120.csv")

After labeling hashtags one at at time, we address the issue of missing hashtags so the subsequent code may be uneccesary:

In [11]:
def find_unlabeled_hashtags(original_df: pd.DataFrame, labeled_df: pd.DataFrame) -> pd.DataFrame:
    # Assuming the hashtag column is named the same in both dataframes
    unlabeled = original_df[~original_df['hashtag'].isin(labeled_df['hashtag'])]
    print(f"Original hashtags: {len(original_df)}")
    print(f"Labeled hashtags: {len(labeled_df)}")
    print(f"Unlabeled hashtags: {len(unlabeled)}")
    
    return unlabeled

def prepare_unlabeled_for_gpt(unlabeled_df: pd.DataFrame) -> str:
    return ', '.join(unlabeled_df['hashtag'].tolist())

In [12]:
unlabeled_df = find_unlabeled_hashtags(df, result_df)

Original hashtags: 5519
Labeled hashtags: 5242
Unlabeled hashtags: 408


In [13]:
duplicated_hashtags = result_df[result_df.duplicated(subset=['hashtag'], keep=False)]

# Sort by hashtag to see the groupings easily
print("Hashtags with multiple themes:")
print(duplicated_hashtags.sort_values('hashtag'))

# To see a cleaner summary:
summary = duplicated_hashtags.groupby('hashtag')['theme'].agg(list)
print("\nSummary of multiple labels:")
print(summary)


Hashtags with multiple themes:
                  theme     hashtag
2372           platform         911
2421               misc         911
2434               misc         911
544   substance effects  blackedout
2375  substance effects  blackedout
...                 ...         ...
476            platform     تابعوني
5045               misc         ضحك
2916               misc         ضحك
2429               misc       ❄️tok
2367           platform       ❄️tok

[181 rows x 2 columns]

Summary of multiple labels:
hashtag
911                                      [platform, misc, misc]
blackedout               [substance effects, substance effects]
boston                                     [location, location]
brand                                    [platform, misc, misc]
cigs                       [tobacco_nicotine, tobacco_nicotine]
                                        ...                    
xylazineawareness    [awareness and advocacy, other substances]
xylazinetesting      [awaren

In [14]:
# manually select the correct theme from the 25 hashtags that were duplicated
summary.to_excel("../data/duplicated_69_hashtags.xlsx")

/var/folders/jj/bt0ykz9n5_16z2gvbybl0tbc0000gn/T/ipykernel_21220/3331842704.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  summary.to_excel("../data/duplicated_69_hashtags.xlsx")


In [15]:
# Remove all instances of duplicated hashtags
result_df = result_df[~result_df['hashtag'].isin(duplicated_hashtags['hashtag'])]
print(f"Shape of results_df after removing duplicates: {result_df.shape}")

Shape of results_df after removing duplicates: (5061, 2)


In [16]:
fixes_df = pd.read_excel("../data/duplicated_25_hashtags.xlsx")
fixes_df['theme'] = fixes_df['theme'].str.strip('[]').str.strip("'")  # Remove [] and quotes

In [17]:
result_df = pd.concat([result_df, fixes_df], ignore_index=True)
result_df = result_df.drop_duplicates(keep='first')

In [18]:
# Verify no more duplicates
new_duplicates = result_df[result_df.duplicated(subset=['hashtag'], keep=False)]
if not new_duplicates.empty:
    print("\nStill have duplicated hashtags:")
    print(new_duplicates.sort_values('hashtag'))
else:
    print("No duplicates found!")


Still have duplicated hashtags:
                       theme                     hashtag
160        health conditions         depressionawareness
5061  awareness and advocacy         depressionawareness
5062  awareness and advocacy      drugpoisoningawareness
3675       health conditions      drugpoisoningawareness
5063  awareness and advocacy  fentanylpoisoningawareness
4254       health conditions  fentanylpoisoningawareness
1390                   humor               funnypictures
5065                    misc               funnypictures
1947  awareness and advocacy     harmreductionspecialist
5068              occupation     harmreductionspecialist
3677       health conditions                hivawareness
5070  awareness and advocacy                hivawareness
2566                   humor                        ohno
5075                    misc                        ohno
453        health conditions          psychosisawareness
5078  awareness and advocacy          psychosisawarenes

In [19]:
# label just the missing 776
missing_hashtag_chunks_df = chunk_list_to_df(unlabeled_df.hashtag, 50)
missing_hashtag_chunks_df["themes"] = process_posts_in_parallel(missing_hashtag_chunks_df["hashtags_str"].to_list(), max_workers=10,model = "gpt-4o-2024-05-13", task = get_theme, client = client)

In [20]:
missing_result_df, missing_errors = process_hashtag_chunks(missing_hashtag_chunks_df)

In [21]:
# see if there are dupes in the second round
duplicated_hashtags2 = missing_result_df[missing_result_df.duplicated(subset=['hashtag'], keep=False)]

# Sort by hashtag to see the groupings easily
print("Hashtags with multiple themes:")
print(duplicated_hashtags2.sort_values('hashtag'))

# To see a cleaner summary:
summary2 = duplicated_hashtags2.groupby('hashtag')['theme'].agg(list)
print("\nSummary of multiple labels:")
print(summary2)

Hashtags with multiple themes:
                      theme           hashtag
323        tobacco_nicotine               cig
306      consumption method               cig
280        other substances          creatine
375        other substances          creatine
221      consumption method              ecig
242        tobacco_nicotine              ecig
328              occupation  frontlineworkers
407              occupation  frontlineworkers
291              occupation  frontlineworkers
322       substance effects        highasfuck
391       substance effects        highasfuck
181        tobacco_nicotine          nicotine
393        tobacco_nicotine          nicotine
376        other substances         piracetam
279        other substances         piracetam
401                   humor        sobermemes
325                   humor        sobermemes
289                   humor        sobermemes
182        tobacco_nicotine       stopsmoking
164      consumption method       stopsmoking
329

In [22]:
# GET THE STILL MISSING SET FROM THE SECOND BATCH
unlabeled_df2 = find_unlabeled_hashtags(unlabeled_df, missing_result_df)

Original hashtags: 408
Labeled hashtags: 430
Unlabeled hashtags: 41


In [23]:
# label just the missing 31
missing_hashtag_chunks_df2 = chunk_list_to_df(unlabeled_df2.hashtag, 50)
missing_hashtag_chunks_df2["themes"] = process_posts_in_parallel(missing_hashtag_chunks_df2["hashtags_str"].to_list(), max_workers=10,model = "gpt-4o-2024-05-13", task = get_theme, client = client)
missing_result_df2, missing_errors2 = process_hashtag_chunks(missing_hashtag_chunks_df2)

In [24]:
# append
result_full = pd.concat([result_df, missing_result_df, missing_result_df2])
result_full = result_full.drop_duplicates(keep='first')

In [25]:
new_duplicates = result_full[result_full.duplicated(subset=['hashtag'], keep=False)]
if not new_duplicates.empty:
    print("\nStill have duplicated hashtags:")
    print(new_duplicates.sort_values('hashtag'))
else:
    print("No duplicates found!")


Still have duplicated hashtags:
                       theme                     hashtag
1046                platform             @octopusdarling
104                     misc             @octopusdarling
323         tobacco_nicotine                         cig
306       consumption method                         cig
160        health conditions         depressionawareness
5061  awareness and advocacy         depressionawareness
3675       health conditions      drugpoisoningawareness
5062  awareness and advocacy      drugpoisoningawareness
242         tobacco_nicotine                        ecig
221       consumption method                        ecig
4254       health conditions  fentanylpoisoningawareness
5063  awareness and advocacy  fentanylpoisoningawareness
1390                   humor               funnypictures
5065                    misc               funnypictures
1947  awareness and advocacy     harmreductionspecialist
5068              occupation     harmreductionspecialis

In [26]:
print("Current entries for @octopusdarling:")
print(result_full[result_full['hashtag'] == '@octopusdarling'])

# Assuming we want to keep 'platform' theme and remove 'misc'
# Remove both instances
result_full = result_full[result_full['hashtag'] != '@octopusdarling']

# Add back the correct one
new_row = pd.DataFrame({
    'hashtag': ['@octopusdarling'],
    'theme': ['misc'] 
})

# Add the correct row back
result_full = pd.concat([result_full, new_row], ignore_index=True)

# Verify the fix
print("\nAfter fix:")
print(result_full[result_full['hashtag'] == '@octopusdarling'])

# Double check no more duplicates
duplicates = result_full[result_full.duplicated(subset=['hashtag'], keep=False)]
print("\nChecking for any remaining duplicates:")
print(duplicates if not duplicates.empty else "No duplicates found!")

Current entries for @octopusdarling:
         theme          hashtag
1046  platform  @octopusdarling
104       misc  @octopusdarling

After fix:
     theme          hashtag
5522  misc  @octopusdarling

Checking for any remaining duplicates:
                       theme                     hashtag
160        health conditions         depressionawareness
453        health conditions          psychosisawareness
1389                   humor               funnypictures
1946  awareness and advocacy     harmreductionspecialist
2565                   humor                        ohno
2759                   humor                  yerawizard
3674       health conditions      drugpoisoningawareness
3676       health conditions                hivawareness
4020                cannabis                       rasta
4253       health conditions  fentanylpoisoningawareness
5060  awareness and advocacy         depressionawareness
5061  awareness and advocacy      drugpoisoningawareness
5062  awareness an

In [27]:
unlabeled_df3 = find_unlabeled_hashtags(result_full, df)

Original hashtags: 5523
Labeled hashtags: 5519
Unlabeled hashtags: 22


In [28]:
# label just the missing 11
missing_hashtag_chunks_df3 = chunk_list_to_df(unlabeled_df3.hashtag, 50)
missing_hashtag_chunks_df3["themes"] = process_posts_in_parallel(missing_hashtag_chunks_df3["hashtags_str"].to_list(), max_workers=10,model = "gpt-4o-2024-05-13", task = get_theme, client = client)
missing_result_df3, missing_errors3 = process_hashtag_chunks(missing_hashtag_chunks_df3)

In [29]:
result_full2 = pd.concat([result_full, missing_result_df3])
# result_full2 = result_full2.drop_duplicates(keep='first')

In [30]:
# export
result_full2.to_csv("../data/hashtags_5core_with_themes.csv")

---
### OLD

In [24]:
# muqi got the errored chunks to parse. gonna make them long
missing = pd.read_csv("../data/themes_missing_parallel.csv")

In [29]:
type(hashtag_chunks_df.themes[0])

str

In [30]:
type(missing.themes[0])

str

In [ ]:
missing_long, missing_long_e = process_hashtag_chunks(missing)

In [39]:
# append to long
results_full = pd.concat([result_df,missing_long])
print(len(set(result_df.hashtag)))
# drop dupes
results_full_unique = results_full.drop_duplicates(subset="hashtag")

13758


In [40]:
results_full_unique.to_csv("../data/themes_with_missing_long.csv")

In [19]:
# experient with 500 posts
# hashtag_sample2 = random.sample(node_list, 500)
# hashtag_chunks_df2 = chunk_list_to_df(hashtag_sample2, 50)
# hashtag_chunks_df2["themes"] = process_posts_in_parallel(hashtag_chunks_df2["hashtags_str"].to_list(), max_workers=10,model = "gpt-4o-2024-05-13", task = get_theme, client = client)
# hashtag_chunks_df2.to_csv("../data/themes_500_parallel.csv")

---
## Experiment with chunking

In [12]:
from get_themes import get_theme
import random
import csv
import json

def chunk_list(input_list, chunk_size):
  """
  Chunks a list into smaller lists of approximately equal size.

  Args:
      input_list: The list to be chunked.
      chunk_size: The desired size of each chunk.

  Returns:
      A list of lists, where each inner list is a chunk of the original list.
  """
  chunks = []
  for i in range(0, len(input_list), chunk_size):
    chunks.append(input_list[i:i + chunk_size])
  return chunks

def chunk_and_analyze_hashtags(hashtag_list, chunks, model, client, chunk_size=50, output_file='../data/hashtags_themes.csv'):
    # Shuffle the list to ensure random sampling
    random.shuffle(hashtag_list)
    
    # Chunk the list into batches 
    # chunks = [hashtag_list[i:i + chunk_size] for i in range(0, len(hashtag_list), chunk_size)]
    
    total_hashtags = 0
    
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['Theme', 'Hashtag'])  # Write header
        
        for i, chunk in enumerate(chunks):
            # Convert chunk to comma-separated string
            print("length of chunk: ",len(chunk))
            chunk_str = ", ".join(chunk)
            
            # Get themes for this chunk
            themes_str = get_theme(chunk_str, model=model, client=client)
            print(themes_str)
            themes = json.loads(themes_str.replace("'", '"'))
            # Write results to CSV and count hashtags
            chunk_hashtag_count = 0
            for theme, hashtags in themes.items():
                for hashtag in hashtags:
                    csvwriter.writerow([theme, hashtag])
                    chunk_hashtag_count += 1
            
            total_hashtags += chunk_hashtag_count
            print(f"Processed chunk {i+1}/{len(chunks)}. Hashtags in this chunk: {chunk_hashtag_count}")
    
    print(f"Total hashtags processed: {total_hashtags}")
    print(f"Results have been written to {output_file}")
    return total_hashtags

In [11]:
hashtag_sample = random.sample(node_list, 100)
chunks = chunk_list(hashtag_sample, 50)
results = chunk_and_analyze_hashtags(hashtag_sample, model="gpt-4o-2024-05-13", client=client)

50


In [ ]:
# for i, themes in enumerate(results):
#     print(f"Themes for chunk {i+1}:")
#     print(themes)
#     print("---")